In [68]:
import sys
import os

sys.path.insert(0,'..')
#!pip install openpyxl
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load excel

In [ ]:
df = pd.read_excel("../data/RB_TC_extract_2022_07_07.xlsx", "magnet data")

df.dropna(how="all").to_csv("mb_features.csv")

# Load processed excel

In [83]:
df = pd.read_csv("../data/MP3_context_data/20230313_RB_processed.csv", index_col=False)

df_processed = df.dropna(subset=['Circuit Family','Circuit Name','timestamp_fgc'])
df_processed['fpa_identifier'] = df_processed.apply(lambda x: f"{x['Circuit Family']}_{x['Circuit Name']}_{int(x['timestamp_fgc'])}", axis=1)

acquisition_summary = df_processed.drop_duplicates(subset=['fpa_identifier'])
acquisition_summary['datetime'] = pd.to_datetime(acquisition_summary['Date (FGC)'])
acquisition_summary['year'] = acquisition_summary['datetime'].apply(lambda x: x.year)
acquisition_summary['month'] = acquisition_summary['datetime'].apply(lambda x: x.month)
acquisition_summary['day'] = acquisition_summary['datetime'].apply(lambda x: x.day)

acquisition_summary = acquisition_summary[['Circuit Family','Circuit Name','timestamp_fgc', "fpa_identifier", "year", "month", "day"]]
print(len(df), len(df_processed), len(acquisition_summary))

4859 4512 923


<ipython-input-83-d9f526848db9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['fpa_identifier'] = df_processed.apply(lambda x: f"{x['Circuit Family']}_{x['Circuit Name']}_{int(x['timestamp_fgc'])}", axis=1)
<ipython-input-83-d9f526848db9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acquisition_summary['datetime'] = pd.to_datetime(acquisition_summary['Date (FGC)'])
<ipython-input-83-d9f526848db9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [38]:
acquisition_summary.year.value_counts(sort =False) 

2009      1
2010      8
2014     30
2015    173
2016     36
2017      5
2018     24
2021    561
2022     85
Name: year, dtype: int64

In [39]:
acquisition_summary[acquisition_summary.year==2022].month.value_counts(sort =False) 

2      2
3     37
4     19
5      2
6      5
7      5
8     11
9      2
10     1
11     1
Name: month, dtype: int64

# look at images

In [62]:
df_aqs = pd.read_excel("../data/20220707_acquisition_summary.xlsx", index_col=False)
df_aqs['fpa_identifier'] = df_aqs.apply(lambda x: f"RB_{x['Circuit Name']}_{int(x['timestamp_fgc'])}", axis=1)

sucessfull_ts = df_aqs.loc[df_aqs["VoltageNQPS.*U_DIODE"] == 1, "fpa_identifier"]

for fpa_identifier in sucessfull_ts.values:
    print(fpa_identifier)
    #os.remove(f"/eos/project/m/ml-for-alarm-system/private/RB_signals/backup/20230313_data_plots_copy/{fpa_identifier}.png")

In [76]:
im_paths = glob.glob("/eos/project/m/ml-for-alarm-system/private/RB_signals/backup/20230313_data_plots/*")
im_fpa_identifier = np.array([s.split("/")[-1][:-4] for s in im_paths])


bad_im_paths = glob.glob("/eos/project/m/ml-for-alarm-system/private/RB_signals/backup/20230313_data_bad_plots/*")
bad_im_fpa_identifier = np.array([s.split("/")[-1][:-4] for s in bad_im_paths])
len(im_fpa_identifier), len(bad_im_fpa_identifier)

(753, 55)

# Write acquisition_summary

In [91]:
acquisition_summary[["timestamp_fgc_available", "U_diode_data_available"]] = 1
acquisition_summary.loc[~acquisition_summary.fpa_identifier.isin(im_fpa_identifier), "U_diode_data_available"] = 0
acquisition_summary["U_diode_data_useable"] = acquisition_summary["U_diode_data_available"]
acquisition_summary.loc[acquisition_summary.fpa_identifier.isin(bad_im_fpa_identifier), "U_diode_data_useable"] = 0 #VoltageNQPS.*U_DIODE

In [88]:
#acquisition_summary.to_csv("20230313_acquisition_summary.csv",index=False)

In [90]:
acquisition_summary.year.unique()

array([2009, 2010, 2014, 2015, 2016, 2017, 2018, 2021, 2022])

In [101]:
df_summary = acquisition_summary.groupby("year").sum()[["timestamp_fgc_available", "U_diode_data_available", "U_diode_data_useable"]]
df_summary.loc["sum",:] = df_summary.sum(axis=0)

In [103]:
df_summary.astype(int)

,timestamp_fgc_available,U_diode_data_available,U_diode_data_useable
year,,,
2009,1,0,0
2010,8,0,0
2014,30,3,1
2015,173,42,5
2016,36,36,32
2017,5,5,5
2018,24,24,21
2021,561,559,553
2022,85,84,81


# Look at unique values

In [ ]:
df.columns.values

In [ ]:
df['FPA Reason'].value_counts()

In [ ]:
df['Type of Quench'].value_counts()

In [ ]:
df['Nr in Q event'].value_counts()

In [ ]:
df['QDS trigger origin'].value_counts() 

In [ ]:
df['Position'].value_counts() 

In [ ]:
df['Analysis performed by'].value_counts()

In [ ]:
df['Comment'].value_counts()[:20]

# Histograms

In [ ]:
df["Date (FGC)"] = pd.to_datetime(df["Date (FGC)"])
df["year"] = df["Date (FGC)"].dt.year.dropna()
df["month"] = df["Date (FGC)"].dt.month.dropna()

In [ ]:
def plot_hist(df, column, ax, bins=None, title=None, range=None, em=True, xlim=None, ylog=False):
    df[column].hist(label="all quenches", ax=ax, bins=bins, align='left', rwidth=0.9, range=range)
    df[df['Type of Quench'] =="GHe propagation"][column].hist(label="secondary quenches (GHe)", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    df[df['Nr in Q event'] == 1][column].hist(label="primary quenches", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    if em:
        df[df['Type of Quench'] =="EM disturbance"][column].hist(label="secondary quenches (EM)", ax=ax, bins=bins, rwidth=0.9, align='left', range=range)
    if xlim:
        ax.set_xlim(xlim)
    if ylog:
        ax.set_yscale("log")
    ax.legend()
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(5,3, figsize=(25,30))
plot_hist(df, column="year", ax=ax[0,0], bins=np.arange(2008, 2023), title="Year of Quench")
plot_hist(df, column="Circuit Name", ax=ax[0,1], bins=8, title="Circuit of Quench")
plot_hist(df, column="I_Q_M", ax=ax[0,2], title="Current of Quench", range=(df["I_Q_M"].min(), df["I_Q_M"].max()))
plot_hist(df, column="I_Q_SM18", ax=ax[1,0], title="SM18 Test Current of Quenched Magnet", range=(df["I_Q_SM18"].min(), df["I_Q_SM18"].max()))
plot_hist(df, column="Short magnet ID", ax=ax[1,1], bins=100, title="Short magnet ID", range=(df["Short magnet ID"].min(), df["Short magnet ID"].max()))
plot_hist(df, column="Manufacturer", ax=ax[1,2], bins=3, title="Manufacturer")
plot_hist(df[df["Outer cable type"].apply(lambda x: isinstance(x, str))], column="Outer cable type", ax=ax[2,0], title="Outer cable type", bins=df[df["Outer cable type"].apply(lambda x: isinstance(x, str))]["Outer cable type"].unique())
plot_hist(df, column="Inner cable type", ax=ax[2,1], title="Inner cable type", em=False)

plot_hist(df, column="V_symm_max", ax=ax[2,2], title="V_symm_max", xlim=(0,2), bins=100, range=(df["V_symm_max"].min(), df["V_symm_max"].max()))
plot_hist(df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index), column="dU_iQPS/dt", ax=ax[3,0], title="dU_iQPS/dt", xlim=(-25,25), bins=100, range=(df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index)["dU_iQPS/dt"].min(), df.drop(df[df["dU_iQPS/dt"].apply(lambda x: isinstance(x, str))].index)["dU_iQPS/dt"].max()))

plot_hist(df.drop(df[df["Quench origin"]=="?"].index), column="Quench origin", ax=ax[3,1], title="Quench origin")
plot_hist(df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index), column="R_DL_max", ax=ax[4,0], title="R_DL_max", xlim=(-5,200), bins=300, range=(df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index)["R_DL_max"].min(), df.drop(df[df["R_DL_max"].apply(lambda x: isinstance(x, str))].index)["R_DL_max"].max()))
plot_hist(df, column="I at R_DL_max", ax=ax[4,1], title="I at R_DL_max", ylog=True, bins=200, range=(df["I at R_DL_max"].min(), df["I at R_DL_max"].max()))